In [1]:
from pathlib import Path

In [25]:
email_dir = Path(Path.cwd().parent, Path('data/raw/maildir'))

In [49]:
ign_dir = sorted(email_dir.glob('**/discussion_threads')) + sorted(email_dir.glob('**/all_documents'))

In [51]:
len(ign_dir)

203